In [ ]:
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/reg_helper.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/fnn_helper.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/draw_nn.py
! wget https://raw.githubusercontent.com/PAE-ITBA-ML-2019/Regularizacion/master/regularization_helper.py
 
! wget https://github.com/PAE-ITBA-ML-2019/Regularizacion/raw/master/data.zip
! unzip data.zip


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from regularization_helper import plot_it, get_simple_dataset, get_polynimial_set, plot_classifier, save_dataset
from sklearn import model_selection
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import optimizers
from fnn_helper import PlotLosses
from keras.callbacks import ModelCheckpoint 
from keras import regularizers

## Cargo y ploteo dataset

In [ ]:
dataset_200 = np.load('200_samples_OK.npy')
X = dataset_200[:,:2]
y = dataset_200[:, 2]
plot_it(X,y)
plt.show()

## Defino el modelo

In [ ]:
def get_two_layer_model_fixed():
    input_shape=2
    output_size=1
    hidden_units= 10
    lr=2
    decay=0.0
    model = Sequential()
    sgd = optimizers.SGD(lr=lr, decay=decay)
    model.add(Dense(hidden_units,input_dim=input_shape,  activation='sigmoid', ))
    model.add(Dense(output_size, 
                    activation='sigmoid', 
                    kernel_initializer='zeros', 
                    name='Salida'
                   ))
    model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

# K-fold Cross Validation
- No tengo un dataset grande
- Pros: Ayuda a validar mejor el modelo
- Cons: Puede llevar tiempo
- Podriamos haber separado el test set y trabajar con el training set para dividirlo

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(10)

In [ ]:
i=0
training_sets = []
for train_index, test_index in skf.split(X, y):
    i=i+1
    print("CV dataset:", i)
    print(train_index.shape, test_index.shape)
    dictionary = {'X_train':X[train_index], 'y_train':y[train_index], 'X_test':X[test_index],'y_test':y[test_index]}
    training_sets.append(dictionary)

## Grafico uno de los train/test sets

In [ ]:
idx = 1
plot_it(training_sets[idx]['X_train'],training_sets[idx]['y_train'])
plt.show()
plot_it(training_sets[idx]['X_test'],training_sets[idx]['y_test'])
plt.show()

## Entreno de manera tradicional uno de los k-folds

In [ ]:
idx = 1
plot_losses = PlotLosses(plot_interval=10, evaluate_interval=None, x_val=training_sets[idx]['X_test'], y_val_categorical=training_sets[idx]['y_test'])

two_layer_model = get_two_layer_model_fixed()
two_layer_model.fit(training_sets[idx]['X_train'], 
          training_sets[idx]['y_train'], batch_size = 25,
          epochs=200, 
          verbose=0, 
          validation_data=(training_sets[idx]['X_test'], training_sets[idx]['y_test']), 
          callbacks=[plot_losses],
         )

## Entreno usando cross_val_scores de SciKitLearn

In [ ]:
from sklearn.model_selection import cross_val_score
# cross_val_score(classifier, X, y, cv, n_jobs=)
# El problema es que el 'classifier' es de sklearn y tenemos uno de Keras
# Por lo tanto hay que utilizar un wrapper
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
classifier = KerasClassifier(get_two_layer_model_fixed, batch_size=25,  
                             epochs=100, verbose=0)
cross_val_scores = cross_val_score(estimator=classifier, X=X, y=y, cv=skf, 
                                   scoring='accuracy') # n_jobs=-1

In [ ]:
cross_val_scores

In [ ]:
cross_val_scores.mean(), cross_val_scores.std()

- low, high accuracy and low, high variance
- high variance --> overfitting

# Ajuste de Hyperparametros (Parameter tunning)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def get_two_layer_model_variable(lr, hidden_units):
    input_shape=2
    output_size=1
    optimizer = optimizers.SGD(lr=lr, decay=0.0)
    model = Sequential()
    model.add(Dense(hidden_units,input_dim=input_shape,  activation='sigmoid', ))
    model.add(Dense(output_size, 
                    activation='sigmoid', 
                    kernel_initializer='zeros', 
                    name='Salida'
                   ))
    model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
classifier_variable = KerasClassifier(build_fn=get_two_layer_model_variable, epochs=100, verbose=0)

In [ ]:
#sgd = optimizers.SGD(lr=2, decay=0.0)
parameters = {
    'batch_size': [25,100], 
    'lr': [2,1],
    'hidden_units': [3,10]
             }

In [ ]:
grid_search = GridSearchCV(estimator=classifier_variable, cv=skf, param_grid=parameters, scoring='accuracy')

In [ ]:
grid_search.fit(X=X, y=y, verbose=1)

In [ ]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
best_parameters

In [ ]:
best_accuracy